#### Explicación Normalización por grupo con `transform`

Cuando aplicamos una función como:

``` python
def mi_funcion(x):
    return (x - x.mean()) / x.std()
```

a un `groupby().transform`, lo que ocurre es:

-   **Centramos** los datos de cada grupo restando la media.
-   **Escalamos** dividiendo entre la desviación estándar del grupo.

El resultado es que **cada grupo queda con media = 0 y desviación
estándar = 1**\
(esto es lo que se llama *normalización* o *estandarización z-score*).




## Ejemplo de resultados con `transform`

En la práctica podemos mostrar tres vistas distintas de un mismo
DataFrame:

``` python
# Mostrar resultados
display(
    df.head(),             # Datos originales (primeras filas)
    df_normalizado.head(), # Datos normalizados (media=0, std=1 dentro de cada departamento)
    df_centrado.head()     # Datos centrados (media=0 dentro de cada departamento)
)
```

### Interpretación del resultado

1.  **Datos originales (`df.head()`)**
    -   Contienen los valores tal como estaban en el DataFrame.\
    -   Cada departamento puede tener medias muy diferentes.
2.  **Datos normalizados (`df_normalizado.head()`)**
    -   Cada grupo (`department`) se transformó con la fórmula:\
        \\(x - `\text{mean}`{=tex}) / `\text{std}`{=tex}\\\
    -   Resultado: media = 0, desviación estándar = 1 en cada grupo.\
    -   Esto permite comparar grupos en una misma escala estandarizada.
3.  **Datos centrados (`df_centrado.head()`)**
    -   Cada grupo se transformó con la fórmula:\
        \\(x - `\text{mean}`{=tex})\\\
    -   Resultado: la media de cada grupo es 0, pero la dispersión
        original se mantiene.\
    -   Útil cuando solo interesa eliminar el "desplazamiento" de la
        media pero no escalar los datos.


✅ En resumen:\
- **Original** → datos tal cual.\
- **Normalizados** → media = 0 y std = 1 por grupo (z-score).\
- **Centrados** → media = 0, pero con la misma escala original del
grupo.

### ¿Por qué aparecen valores como `-0.707107` y `0.707107`?

Esto sucede porque los grupos tienen **muy pocos elementos**:

-   Si un grupo tiene **2 valores**, después de normalizar siempre
    quedan en `-0.707107` y `+0.707107`.\
    Esto ocurre porque con 2 puntos, la distancia a la media es
    simétrica y la desviación estándar se calcula con `n-1` (muestral).

-   Si un grupo tiene **3 valores equiespaciados** (ejemplo:
    `[10, 20, 30]`), los valores normalizados serían `[-1.0, 0.0, 1.0]`.

-   Si un grupo tiene más valores, los resultados se distribuyen
    alrededor de 0, con mayor o menor dispersión, pero siempre
    cumpliendo:

    -   Media del grupo = 0
    -   Desviación estándar del grupo = 1



### Interpretación del resultado

Cuando ves un `head` con valores como:

    0   -0.707107   0.707107
    1   -0.707107  -0.707107
    2   -0.707107  -0.707107
    3    0.707107  -0.707107
    4    0.707107   0.707107

significa que:

-   Cada columna corresponde a una variable numérica en el DataFrame.
-   Cada grupo (`department`) se normalizó por separado.
-   Los valores `±0.707107` aparecen porque los grupos tienen pocos
    registros y la fórmula de la desviación estándar ajusta los valores
    a esa escala.



✅ En resumen: - `mean / std` → centrar y escalar dentro de cada
grupo. - Los valores dependen del tamaño del grupo, por eso aparecen
`±0.707107` cuando hay 2 elementos.

### ¿Por qué en el centrado aparecen valores como `-0.5`, `0.5`, `1.0`, `-3.5`, etc.?

Cuando usamos:

``` python
df.groupby("department").transform(lambda x: x - x.mean())
```

lo que hacemos es **centrar los datos de cada grupo**: - A cada valor se
le resta la **media del grupo** al que pertenece. - El resultado
conserva la misma forma que el DataFrame original.



### Ejemplo del resultado observado

       anio   VV
    0  -0.5   1.0
    1  -0.5  -3.5
    2  -0.5  -3.0
    3   0.5  -1.0
    4   0.5   3.5



### Interpretación

1.  **Columna `anio`**
    -   Los valores quedaron en `-0.5` y `0.5`.
    -   Esto ocurre porque, en cada grupo (`department`), se resta la
        media de los años.
    -   Ejemplo: si en un grupo los años eran `[2020, 2021]`,\
        la media es `2020.5`.
        -   `2020 - 2020.5 = -0.5`\
        -   `2021 - 2020.5 = +0.5`
2.  **Columna `VV`**
    -   Cada valor refleja su desviación respecto a la media de `VV` en
        su grupo.\
    -   Ejemplo: si la media de `VV` en un grupo es `10.5`:
        -   Para un registro con `VV = 14`: `14 - 10.5 = 3.5`\
        -   Para un registro con `VV = 7`: `7 - 10.5 = -3.5`
  
### ✅ En resumen

-   En `anio`, el centrado muestra **diferencias relativas al promedio
    de años del grupo**.\
-   En `VV`, el centrado muestra **cuánto se aleja cada valor de la
    media de su grupo**.\
-   El resultado es correcto y esperado: tras centrar, cada grupo tiene
    **media = 0**, pero mantiene su dispersión original.